# CrateDB Chat Message History

This notebook demonstrates how to use the `CrateDBChatMessageHistory`
to manage chat history in CrateDB, for supporting conversational memory.

## Prerequisites

In [ ]:
!#pip install langchain sqlalchemy-cratedb

## Configuration

To use the storage wrapper, you will need to configure two details.

1. Session Id - a unique identifier of the session, like user name, email, chat id etc.
2. Database connection string: An SQLAlchemy-compatible URI that specifies the database
   connection. It will be passed to SQLAlchemy create_engine function.

In [52]:
from langchain.memory.chat_message_histories import CrateDBChatMessageHistory

CONNECTION_STRING = "crate://crate@localhost:4200/?schema=example"

chat_message_history = CrateDBChatMessageHistory(
    session_id="test_session", connection_string=CONNECTION_STRING
)

## Basic Usage

In [53]:
chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

In [61]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='Hi', additional_kwargs={}, example=False)]

## Custom Storage Model

The default data model, which stores information about conversation messages only
has two slots for storing message details, the session id, and the message dictionary.

If you want to store additional information, like message date, author, language etc.,
please provide an implementation for a custom message converter.

This example demonstrates how to create a custom message converter, by implementing
the `BaseMessageConverter` interface.

In [55]:
from datetime import datetime
from typing import Any

import sqlalchemy as sa
from langchain.memory.chat_message_histories.sql import BaseMessageConverter
from langchain.schema import AIMessage, BaseMessage, HumanMessage, SystemMessage
from sqlalchemy.orm import declarative_base

Base = declarative_base()


class CustomMessage(Base):
    __tablename__ = "custom_message_store"

    id = sa.Column(sa.BigInteger, primary_key=True, server_default=sa.func.now())
    session_id = sa.Column(sa.Text)
    type = sa.Column(sa.Text)
    content = sa.Column(sa.Text)
    created_at = sa.Column(sa.DateTime)
    author_email = sa.Column(sa.Text)


class CustomMessageConverter(BaseMessageConverter):
    def __init__(self, author_email: str):
        self.author_email = author_email

    def from_sql_model(self, sql_message: Any) -> BaseMessage:
        if sql_message.type == "human":
            return HumanMessage(
                content=sql_message.content,
            )
        elif sql_message.type == "ai":
            return AIMessage(
                content=sql_message.content,
            )
        elif sql_message.type == "system":
            return SystemMessage(
                content=sql_message.content,
            )
        else:
            raise ValueError(f"Unknown message type: {sql_message.type}")

    def to_sql_model(self, message: BaseMessage, session_id: str) -> Any:
        now = datetime.now()
        return CustomMessage(
            session_id=session_id,
            type=message.type,
            content=message.content,
            created_at=now,
            author_email=self.author_email,
        )

    def get_sql_model_class(self) -> Any:
        return CustomMessage


if __name__ == "__main__":
    Base.metadata.drop_all(bind=sa.create_engine(CONNECTION_STRING))

    chat_message_history = CrateDBChatMessageHistory(
        session_id="test_session",
        connection_string=CONNECTION_STRING,
        custom_message_converter=CustomMessageConverter(
            author_email="test@example.com"
        ),
    )

    chat_message_history.add_user_message("Hello")
    chat_message_history.add_ai_message("Hi")

In [60]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='Hi', additional_kwargs={}, example=False)]

## Custom Name for Session Column

The session id, a unique token identifying the session, is an important property of
this subsystem. If your database table stores it in a different column, you can use
the `session_id_field_name` keyword argument to adjust the name correspondingly.

In [57]:
import json
import typing as t

from langchain.memory.chat_message_histories.cratedb import CrateDBMessageConverter
from langchain.schema import _message_to_dict

Base = declarative_base()


class MessageWithDifferentSessionIdColumn(Base):
    __tablename__ = "message_store_different_session_id"
    id = sa.Column(sa.BigInteger, primary_key=True, server_default=sa.func.now())
    custom_session_id = sa.Column(sa.Text)
    message = sa.Column(sa.Text)


class CustomMessageConverterWithDifferentSessionIdColumn(CrateDBMessageConverter):
    def __init__(self):
        self.model_class = MessageWithDifferentSessionIdColumn

    def to_sql_model(self, message: BaseMessage, custom_session_id: str) -> t.Any:
        return self.model_class(
            custom_session_id=custom_session_id,
            message=json.dumps(_message_to_dict(message)),
        )


if __name__ == "__main__":
    Base.metadata.drop_all(bind=sa.create_engine(CONNECTION_STRING))

    chat_message_history = CrateDBChatMessageHistory(
        session_id="test_session",
        connection_string=CONNECTION_STRING,
        custom_message_converter=CustomMessageConverterWithDifferentSessionIdColumn(),
        session_id_field_name="custom_session_id",
    )

    chat_message_history.add_user_message("Hello")
    chat_message_history.add_ai_message("Hi")

In [58]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='Hi', additional_kwargs={}, example=False)]